# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [5]:
# Number of samples of black and white sounding names
n_white = (data['race'] == 'w').sum()
n_black = (data['race'] == 'b').sum()
print n_white, n_black

2435 2435


#### Answer 1:
We have a sample which constructed of black-sounding names and white-sounding names, so we can perform a two-sample bootstrap hypothesis test for difference of means, in order to determine if there is a difference in means (proportions) of callbacks, for these two populations. Each of the samples has a size of 2435, so we can assume that the CLT holds.

#### Answer 2:
The null hypothesis is that the proportions of callbacks for black-sounding names and white-sounding names are equal, or alternatively, that the difference between the proportions is 0. The alternate hypothesis is that the proportion of callbacks for black-sounding names is different from the proportion of callbacks for white-sounding names.

In [98]:
def draw_bs_reps(data, func, size=1):
    """Draw bootstrap replicates."""

    # Initialize array of replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = func(np.random.choice(data, size=len(data)))

    return bs_replicates

In [99]:
black_sample = data['call'].loc[data['race'] == 'b']
white_sample = data['call'].loc[data['race'] == 'w']

mean_black = np.mean(black_sample)
mean_white = np.mean(white_sample)

# Compute difference of proportions of callbacks
empirical_diff_means = mean_white - mean_black

print 'proportion of callbacks for black-sounding names: %f' %mean_black
print 'proportion of callbacks for white-sounding names: %f' %mean_white
print 'difference in proportions of callbacks: %f' %empirical_diff_means

proportion of callbacks for black-sounding names: 0.064476
proportion of callbacks for white-sounding names: 0.096509
difference in proportions of callbacks: 0.032033


In [101]:
# Compute mean of all callbacks
mean_all = np.mean(data['call'])

# Generate shifted arrays
white_shifted = white_sample - mean_white + mean_all
black_shifted = black_sample - mean_black + mean_all 

# Compute 100,000 bootstrap replicates from shifted arrays
bs_replicates_white = draw_bs_reps(white_shifted, np.mean, 100000)
bs_replicates_black = draw_bs_reps(black_shifted, np.mean, 100000)

# Get replicates of difference of means
bs_replicates = bs_replicates_white - bs_replicates_black

# Compute and print p-value: p
p = np.sum(bs_replicates >= empirical_diff_means) / 100000.0
print 'p-value: %f' %p

p-value: 0.000030


In [115]:
bs_white = draw_bs_reps(white_sample, np.mean, 100000)
bs_black = draw_bs_reps(black_sample, np.mean, 100000)

# Get replicates of difference of means
bs_replicates = bs_white - bs_black

conf_int = np.percentile(bs_replicates,[2.5,97.5])
se = np.std(bs_replicates)

print '95 percent confidence interval: [%f,%f]' %(conf_int[0],conf_int[1])
print 'Margin of error: %f' %(2*se)

95 percent confidence interval: [0.016838,0.047228]
Margin of error: 0.015630


#### Answer 3:
The margin of error for the difference in proportions is 0.0156, the 95% confidence interval is [0.0168,0.0472], and the p-value is 0.00003

#### Answer 4:
After performing an hypothesis test, we can conclude that it is unlikely that the proportion of callbacks to white-sounding names is equal to the proportion of callbacks to black-sounding names. In the simulation I did, I received that only 3 out of 100,000 resamples, had a difference in means as extreme or higher then the one in the original sample, under the assumption that the difference in means is in fact 0. According to these findings, we can claim with high confidence, that the original difference in means is significantly different from 0, which means that race has a significant impact on the rate of callbacks for resumes.

#### Answer 5:
The analysis does not mean that race/name is the most important factor in callback success, because we only tested this parameter. In order to determine whether there is a more important factor in callback success, I would check for correlation between success and other factors, and the factor with the highest correlation with success (in absolute value) would be the most important factor in callback success.